# Using a mix of stats and sentance similar to predict course purchases

# Read file

In [ ]:
import pandas as pd
import jsonlines
import numpy as np
import csv
from tqdm import tqdm, trange
import sys
sys.path.append("..")
from utils import *

from sentence_transformers import SentenceTransformer,util
tokenizer = SentenceTransformer('shibing624/text2vec-base-chinese')
user = load_from_pickle('../data/user')
course = load_from_pickle('../data/course')

df_test = pd.read_csv("../data/test_unseen.csv")
df_test = convert_test(df_test,user)
de = df_test['text'].to_list()
eval_embeddings = [tokenizer.encode(i) for i in tqdm(de)]
course_list = course.course_name.to_list()



# set the candidate course by course_filter result
* get target course embeddings

In [ ]:
candidate = list(set(
       [686, 699, 501, 500, 502, 656, 665]
       ))
course_top = [course_list[i] for i in candidate]
course_embeddings = [tokenizer.encode(i) for i in tqdm(course_top)]

# predict 

In [ ]:
ans = []
for i,k in tqdm(enumerate(eval_embeddings)):
    prob_index = util.pytorch_cos_sim(eval_embeddings[i],course_embeddings).squeeze(0).argsort(descending=True).tolist()
    prob = [candidate[i] for i in prob_index]
    ans.append(list(dict.fromkeys(prob))[:])
ans_ = [list(map(lambda x:list(course.index)[x],ans[i])) for i in range(len(ans))]

# delete seen course from predict

In [ ]:
seen = load_from_pickle('../data/train')
seen = {i:k.split(' ') for i,k in zip(seen.user_id,seen.course_id)}
for i,user in enumerate(df_test['user_id'].values):
    if(user in seen.keys()):
        ans_[i] = [x for x in ans_[i] if x not in seen[user]]
ans_ = [" ".join(item) for item in ans_]

# Write to csv

In [ ]:
data = {
    'user_id':df_test['user_id'].to_list(),
    'course_id':ans_
}
pd.DataFrame(data).to_csv('./top7.csv',index=False)
